In [85]:
import pandas as pd
import numpy as np
import re
import string

In [74]:
data = pd.read_csv('data/master.csv')

In [67]:
#### MOVE TO DATA CLEANUP!
def remove_link(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

In [69]:
data['Comment'] = data['Comment'].apply(lambda x: remove_link(str(x)))

In [77]:
data['EditorPick'] = data['EditorPick'].apply(lambda x: 0 if x == 'False' else x)
data['EditorPick'] = data['EditorPick'].apply(lambda x: 1 if x == 'True' else x)
data['EditorPick'] = data['EditorPick'].astype(float)

In [78]:
data['Recommendations'] = data['Recommendations'].apply(lambda x: 0 if type(x) == str else x)

In [83]:
def word_count(text):
    out = text.translate(string.maketrans("",""), string.punctuation)
    return len(out.split())

In [89]:
data['word_count'] = data['Comment'].apply(lambda x: word_count(str(x)))

In [90]:
data.head()

,Unnamed: 0,Comment,URL,Recommendations,Locations,EditorPick,userID,date,Sentiment,Sentiment_sent,Sentiment_b,Sentiment_b_sent,Subjectivity,Subjectivity_sent,Subjectivity_b,Subjectivity_b_sent,Candidate,State,City,word_count
0,0,What Republican wannabe POTUS candidate among ...,http://takingnote.blogs.nytimes.com/2015/09/03...,0,"West Palm Beach, FL",0,33461964,1441452450,-0.066911,-0.200,0,0,0.471123,-0.200,1,0,Hilary,FL,West Palm Beach,141
1,1,"Tax Pledges, loyalty pledges but still no pled...",http://takingnote.blogs.nytimes.com/2015/09/03...,0,mt dora,0,64318277,1441411769,0.200000,0.200,1,1,0.300000,0.200,1,1,Hilary,NaN,Mt Dora,25
2,2,"Correct me if I'm wrong, but I believe Republi...",http://takingnote.blogs.nytimes.com/2015/09/03...,0,Boston,0,67000772,1441397419,-0.025000,-0.025,0,0,0.383333,-0.025,1,0,Hilary,NaN,Boston,53
3,3,Trump isn't running for President he is runnin...,http://takingnote.blogs.nytimes.com/2015/09/03...,0,Virginia,0,25933269,1441388186,0.300000,0.300,1,1,0.200000,0.300,1,1,Hilary,VA,NaN,17
4,4,The GOP now owns Trump and all his narrow-mind...,http://takingnote.blogs.nytimes.com/2015/09/03...,0,Arizona,0,36052407,1441389161,0.000000,0.000,1,1,0.000000,0.000,1,1,Hilary,AZ,NaN,29


In [93]:
reg = data.drop(['Unnamed: 0','URL', 'Comment','Sentiment', 'Sentiment_b', 'Subjectivity', 
             'Subjectivity_b', 'Locations','Candidate', 'userID', 
             'date', 'State','City'], axis = 1)

In [95]:
X = reg.drop('Recommendations', axis = 1).values
Y = reg['Recommendations'].values

In [98]:
len(Y)

140376

In [96]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3)

ValueError: Found arrays with inconsistent numbers of samples: [     5 140376]